<a href="https://colab.research.google.com/github/Elwing-Chou/tibame0410/blob/main/tibame_0430_night.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import urllib.request as req
import bs4 as bs

def save_post_meta(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())
    # 作者/看版/標題/日期
    metas = html.find_all("span", {"class":"article-meta-value"})
    uid_text = metas[0].text
    board_text = metas[1].text
    title_text = metas[2].text
    time_text = metas[3].text
    # print(uid_text, board_text, title_text, time_text, sep="\n")
    # 每個推虛文 [{"type":"", "id":"", "text":"", "date_ip":""}]
    # JSON格式: {"左者": "看板": "表提": "推虛文":[]}
    push_list = []
    pushes = html.find_all("div", {"class":"push"})
    for push in pushes:
        push_meta = push.find_all("span")
        # 推噓處理
        push_type_text = push_meta[0].text
        if "推" in push_type_text:
            push_type_text = 1
        elif "噓" in push_type_text:
            push_type_text = -1
        else:
            push_type_text = 0
        # 推噓id
        push_id_text = push_meta[1].text
        # 推噓內容
        push_content_text = push_meta[2].text.replace(": ", "")
        # 推噓ipdate
        push_ipdate_text = push_meta[3].text.strip()
        # print(push_type_text, push_id_text, push_content_text, push_ipdate_text, sep="\t")
        push_data = {
            "type":push_type_text,
            "id":push_id_text,
            "text":push_content_text,
            "date_ip":push_ipdate_text
        }
        push_list.append(push_data)
    # print(push_list)
    post = {
        "作者":uid_text,
        "看板":board_text,
        "標題":title_text,
        "日期":time_text,
        "推噓文":push_list,
    }
    # print(post)
    fn = url.split("/")[-1] + ".json"
    with open(fn, "w", encoding="utf-8") as f:
        json.dump(post, f, ensure_ascii=False, indent=4)



```
當你發現你沒有class/id做篩選的時候
1. 找他老爸
2. 自己想辦法篩選
```



In [ ]:
import json
import urllib.request as req
import bs4 as bs
import os

def save_post_img(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    # prepare directory
    dn = url.split("/")[-1]
    if not os.path.exists(dn):
        os.makedirs(dn)

    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())

    allow_sub = ["jpg", "jpeg", "png", "gif"]
    link_list = html.find_all("a")
    for link in link_list:
        link_href = link["href"]
        sub = link_href.split(".")[-1]
        # lower是字串的專屬功能
        if sub.lower() in allow_sub:
            img_url = link_href
            # 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
            r = req.Request(img_url)
            r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
            img_resp = req.urlopen(r)
            img_content = img_resp.read()
            # 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
            # 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
            #                              wb rb -> for 非純文字
            fn = dn + "/" + img_url.split("/")[-1]
            with open(fn, "wb") as f:
                f.write(img_content)

In [ ]:
# os.path這件事我們後面會解釋
import os
# 如果資料夾不存在, 就創起來
if not os.path.exists("a/b"):
    os.makedirs("a/b")

img_url = "https://i.imgur.com/S20cVi5.jpeg"
# 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
r = req.Request(img_url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
img_resp = req.urlopen(r)
img_content = img_resp.read()
# 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
# 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
#                              wb rb -> for 非純文字
with open("a/b/test.png", "wb") as f:
    f.write(img_content)

In [ ]:
import json
import urllib.request as req
import bs4 as bs
import os

url = "https://www.ptt.cc/bbs/Beauty/index4000.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
resp = req.urlopen(r)
html = bs.BeautifulSoup(resp.read())

div_list = html.find_all("div", {"class":"title"})
for div in div_list:
    post_link = div.find("a")
    if not post_link == None:
        post_link_url = "https://www.ptt.cc" + post_link["href"]
        print(post_link_url)
        save_post_meta(post_link_url)
        save_post_img(post_link_url)

In [ ]:
# 如果你要把雲端硬碟當成隨身碟
dn = "drive/MyDrive/tiba0430"
if not os.path.exists(dn):
    os.makedirs(dn)
fn = dn + "/" + "a.txt"
with open(fn, "w", encoding="utf-8") as f:
    f.write("abcde")

In [ ]:
!zip -r /content/file.zip /content/M.1745582404.A.011.html

In [26]:
import glob
import json
import pandas as pd
# [{}, {}, {}]
table = []
for fn in glob.glob("M.*.json"):
    with open(fn, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
        table.append(data)

df = pd.json_normalize(table)
df

,作者,看板,標題,日期,推噓文
0,clbkan (行動代號龍捲風),Beauty,[神人] 東森廣告看到的,Mon Apr 28 10:47:22 2025,"[{'type': 0, 'id': 'nolimits', 'text': '感覺就AI ..."
1,ReiKuromiya (ReiKuromiya),Beauty,[正妹] 日本橄欖球啦啦隊,Mon Apr 28 20:27:19 2025,"[{'type': 1, 'id': 'mfuqq', 'text': '這才叫啦啦隊', ..."
2,Gentlemon (肥宅紳士),Beauty,[正妹] Cosplay 1901 日本 瑪琳 莉莉艾露 二乃,Mon Apr 28 18:22:12 2025,"[{'type': 0, 'id': 'sothyengiua', 'text': '可',..."
3,yokann (PTT和平大使),Beauty,[正妹] 神木麗,Mon Apr 28 04:17:14 2025,"[{'type': 1, 'id': 'iamseanli', 'text': '阿麗可甜可..."
4,FWMK2RC6RS (BIGTITCR),Beauty,[正妹] Roxy Campbell 36G,Mon Apr 28 16:44:59 2025,"[{'type': 1, 'id': 'prostitute', 'text': '就是UU..."
5,CavendishJr (梅西務實梅西可愛),Beauty,[正妹] 上坂すみれ,Mon Apr 28 13:22:38 2025,"[{'type': 1, 'id': 'coffeedemon', 'text': '大政委..."
6,Mikasa40555 (米卡沙),Beauty,[神人] 網路廣告看到的妹子,Mon Apr 28 19:30:01 2025,"[{'type': 0, 'id': 'Cuchulainn', 'text': '張小壹'..."
7,iwjwlmsc0707 (JEFF HSU),Beauty,[正妹] 珠珠寶貝,Sun Apr 27 22:25:53 2025,"[{'type': 1, 'id': 'Dubufan0528', 'text': '我婆'..."
8,yokann (PTT和平大使),Beauty,[正妹] 科技業,Mon Apr 28 04:27:00 2025,"[{'type': -1, 'id': 'louis5265', 'text': '準備找科..."
9,YMGee (紀縈梅),Beauty,Re: [正妹] 髮型設計師,Mon Apr 28 21:22:25 2025,"[{'type': 1, 'id': 'hanainori', 'text': '魅惑美容室..."
